In [549]:
import requests
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np 

# Foursquare

In [550]:
load_dotenv()
key_4square = os.getenv("FOURSQUARE_KEY")

(I've put the full list of what each location code means in a plaintext because it is rather long.)

In [551]:
poi_list = "10002,10027,10051,10056,12065,12066,12080,15014,16003,16005,16006,16011,16018,16025,16026,16032,16043,17104,17105,17114,19028,19031,19042,13003,13028,13061,13062,13065,13386,13387"


In [552]:
headers = {"Accept": "application/json"}
headers['Authorization'] = key_4square

I used Postman to figure out the query structure, because I wasn't finding any how-to's _anywhere_ and I didn't want to waste my calls, and I remembered doing one in there during class. <p>
I ran a test with the first bike station to make sure that a) it worked and b) I wasn't going to get a ridiculous amount of returns (testing and testcall) before working on building an iterative way to do repeated calls and have them all joined together.

In [553]:
dummy = ['54.607397,-5.937086','54.597177,-5.930825']

In [554]:
# storage = []
# for string in dummy:
#     url = "https://api.foursquare.com/v3/places/search?ll=" + string + '&radius=1000&categories=' + poi_list
#     result = requests.get(url, headers=headers)
#     storage.append(result.json())
    

**Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice.**

In [555]:
latlong_list = ['54.607397,-5.937086','54.597177,-5.930825','54.588123,-5.935525',
'54.589329,-5.933508','54.594821,-5.933116','54.592172,-5.925547','54.603420,-5.906825',
'54.586112,-5.891912','54.572534,-5.915076','54.599697,-5.936308','54.604986,-5.932853',
'54.609674,-5.972700','54.589851,-5.958463','54.586848,-5.939333','54.611906,-5.902814',
'54.599358,-5.929838','54.605713,-5.911299','54.578142,-5.918787','54.592680,-5.931528',
'54.595315,-5.949091','54.595255,-5.923680','54.581962,-5.977871','54.601425,-5.925847',
'54.583948,-5.936989','54.597270,-5.921116','54.599399,-5.947595','54.583901,-5.932810',
'54.595840,-5.929881','54.618475,-5.938534','54.597573,-5.927663','54.621307,-5.923744',
'54.590889,-5.967314','54.601319,-5.931308','54.612417,-5.936748','54.594321,-5.915236',
'54.608528,-5.941029','54.598753,-5.899396','54.591219,-5.922650','54.580556,-5.920504',
'54.596086,-5.892781','54.598203,-5.924033','54.577839,-5.881805','54.603687,-5.944403',
'54.583056,-5.956993','54.598305,-5.890582','54.600337,-5.905516','54.594131,-5.868545',
'54.594634,-5.927441','54.601437,-5.921996','54.574854,-5.932134','54.603923,-5.917243',
'54.593469,-5.934409','54.611849,-5.942139','54.595321,-5.916460','54.597142,-5.934827',
'54.575364,-5.955373','54.576116,-5.942574']

In [556]:
# storage = []
# for string in latlong_list:
#     url = "https://api.foursquare.com/v3/places/search?ll=" + string + '&radius=1000&categories=' + poi_list
#     result = requests.get(url, headers=headers)
#     storage.append(result.json())

In [557]:
import json 

In [558]:
# with open('retrieval_4square.json', 'w') as f:
#     json.dump(storage, f)

Saved all of that so I don't have to redo the call if I have to close the notebook. 

In [559]:
pd.read_json('retrieval_4square.json').head(2)

,results,context
0,"[{'fsq_id': '50ca5592e4b0cc5bad7c6449', 'categ...",{'geo_bounds': {'circle': {'center': {'latitud...
1,"[{'fsq_id': '5ae825e4ad910e002c72a002', 'categ...",{'geo_bounds': {'circle': {'center': {'latitud...


The functions I tried to write to unnest this broke and shuffled the categories around really badly, so now I am doing it manually before I sort the tags.<p>
The functions that broke:
```
def locationgroups(dataframe):
    station_number = list(range(0, 57))
    step1 = pd.json_normalize(dataframe[0])
    step2 = pd.json_normalize(dataframe[1])
    step3 = pd.json_normalize(dataframe[2])
    step4 = pd.json_normalize(dataframe[3])
    step5 = pd.json_normalize(dataframe[4])
    step6 = pd.json_normalize(dataframe[5])
    step7 = pd.json_normalize(dataframe[6])
    step8 = pd.json_normalize(dataframe[7])
    step9 = pd.json_normalize(dataframe[8])
    step10 = pd.json_normalize(dataframe[9])

    step1.insert(loc = 0,column = 'station_num',value = station_number)
    step2.insert(loc = 0,column = 'station_num',value = station_number)
    step3.insert(loc = 0,column = 'station_num',value = station_number)
    step4.insert(loc = 0,column = 'station_num',value = station_number)
    step5.insert(loc = 0,column = 'station_num',value = station_number)
    step6.insert(loc = 0,column = 'station_num',value = station_number)
    step7.insert(loc = 0,column = 'station_num',value = station_number)
    step8.insert(loc = 0,column = 'station_num',value = station_number)
    step9.insert(loc = 0,column = 'station_num',value = station_number)
    step10.insert(loc = 0,column = 'station_num',value = station_number)

    return pd.concat([
        step1,step2,step3,step4,step5,step6,
        step7,step8,step9,step10]).sort_values('station_num')
```
```
def get_cat(dataframe):
    categories = pd.concat([
        pd.json_normalize(pd.json_normalize(layer1['categories'])[0])[['name']].rename(columns={"name":"tag_1"}),
        pd.json_normalize(pd.json_normalize(layer1['categories'])[1])[['name']].rename(columns={"name":"tag_2"}),
        pd.json_normalize(pd.json_normalize(layer1['categories'])[2])[['name']].rename(columns={"name":"tag_3"}),
        pd.json_normalize(pd.json_normalize(layer1['categories'])[3])[['name']].rename(columns={"name":"tag_4"}),
        pd.json_normalize(pd.json_normalize(layer1['categories'])[4])[['name']].rename(columns={"name":"tag_5"}),
        pd.json_normalize(pd.json_normalize(layer1['categories'])[5])[['name']].rename(columns={"name":"tag_6"}),
        pd.json_normalize(pd.json_normalize(layer1['categories'])[6])[['name']].rename(columns={"name":"tag_7"})],axis=1)
    return categories
    ```

**Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)**

In [560]:
pd.json_normalize(pd.read_json('retrieval_4square.json')['context']).head(4)

,geo_bounds.circle.center.latitude,geo_bounds.circle.center.longitude,geo_bounds.circle.radius
0,54.607397,-5.937086,1000
1,54.597177,-5.930825,1000
2,54.588123,-5.935525,1000
3,54.589329,-5.933508,1000


In [561]:
returned = pd.read_json('retrieval_4square.json')

In [562]:
cleaning = pd.concat([returned,pd.json_normalize(returned['context'])],axis=1)

In [563]:
cleaning = pd.concat([cleaning,pd.json_normalize(cleaning['results'])],axis=1)
cleaning.head(3)

,results,context,geo_bounds.circle.center.latitude,geo_bounds.circle.center.longitude,geo_bounds.circle.radius,0,1,2,3,4,5,6,7,8,9
0,"[{'fsq_id': '50ca5592e4b0cc5bad7c6449', 'categ...",{'geo_bounds': {'circle': {'center': {'latitud...,54.607397,-5.937086,1000,"{'fsq_id': '50ca5592e4b0cc5bad7c6449', 'catego...","{'fsq_id': '50b0e62be4b0441cd92ec4f3', 'catego...","{'fsq_id': '520080a3498eb97026f147b8', 'catego...","{'fsq_id': '4b64a08bf964a52082c32ae3', 'catego...","{'fsq_id': '5a1fedbcbcbf7a12b589f1f9', 'catego...","{'fsq_id': '52988e8511d22448bd236206', 'catego...","{'fsq_id': '4bd352d741b9ef3b0fe3ffe5', 'catego...","{'fsq_id': '4c1669aea5eb76b040ccc4b7', 'catego...","{'fsq_id': '4bb897143db7b7135e65219a', 'catego...","{'fsq_id': '4b06f46df964a5201af422e3', 'catego..."
1,"[{'fsq_id': '5ae825e4ad910e002c72a002', 'categ...",{'geo_bounds': {'circle': {'center': {'latitud...,54.597177,-5.930825,1000,"{'fsq_id': '5ae825e4ad910e002c72a002', 'catego...","{'fsq_id': '4b64a08bf964a52082c32ae3', 'catego...","{'fsq_id': '4fc21ef5e4b099ce175faf24', 'catego...","{'fsq_id': '4bb3c394715eef3b6bce86bb', 'catego...","{'fsq_id': '5a37b9bd16fa04126cd4ccd7', 'catego...","{'fsq_id': '4bd352d741b9ef3b0fe3ffe5', 'catego...","{'fsq_id': '4c4647bd342c1b8d3040b689', 'catego...","{'fsq_id': '4b459b93f964a5207d0e26e3', 'catego...","{'fsq_id': '4b071106f964a52093f622e3', 'catego...","{'fsq_id': '5a1fedbcbcbf7a12b589f1f9', 'catego..."
2,"[{'fsq_id': '545b434b498e9a85bdb35967', 'categ...",{'geo_bounds': {'circle': {'center': {'latitud...,54.588123,-5.935525,1000,"{'fsq_id': '545b434b498e9a85bdb35967', 'catego...","{'fsq_id': '56f3e2b5498e6346b7f91b31', 'catego...","{'fsq_id': '4c39a7270a71c9b6e7d642c9', 'catego...","{'fsq_id': '4b7554a1f964a520ac062ee3', 'catego...","{'fsq_id': '4c191a0c4ff90f475e6b0f49', 'catego...","{'fsq_id': '4b459b93f964a5207d0e26e3', 'catego...","{'fsq_id': '4b7d734ff964a52022bf2fe3', 'catego...","{'fsq_id': '4bf0422df831c92849cd01f2', 'catego...","{'fsq_id': '504b9545e4b0b18004d43314', 'catego...","{'fsq_id': '4b65d319f964a5203a022be3', 'catego..."


In this table, each row corresponds to a station, and each column to a P.O.I., but to unnest them, each column will become a row, meaning each new row will need to  be tagged with which row it initially belonged to, in a way that **can't** get lost with transformations.

In [564]:
station_num = [*range(1,58,1)]


In [565]:
cleaning['station_num'] = station_num
cleaning = cleaning.drop(['results','context','geo_bounds.circle.radius'],axis=1)
cleaning = cleaning.rename(columns={'geo_bounds.circle.center.latitude': 'station_latitude', 'geo_bounds.circle.center.longitude':'station_longitude'})
cleaning.head(5)

,station_latitude,station_longitude,0,1,2,3,4,5,6,7,8,9,station_num
0,54.607397,-5.937086,"{'fsq_id': '50ca5592e4b0cc5bad7c6449', 'catego...","{'fsq_id': '50b0e62be4b0441cd92ec4f3', 'catego...","{'fsq_id': '520080a3498eb97026f147b8', 'catego...","{'fsq_id': '4b64a08bf964a52082c32ae3', 'catego...","{'fsq_id': '5a1fedbcbcbf7a12b589f1f9', 'catego...","{'fsq_id': '52988e8511d22448bd236206', 'catego...","{'fsq_id': '4bd352d741b9ef3b0fe3ffe5', 'catego...","{'fsq_id': '4c1669aea5eb76b040ccc4b7', 'catego...","{'fsq_id': '4bb897143db7b7135e65219a', 'catego...","{'fsq_id': '4b06f46df964a5201af422e3', 'catego...",1
1,54.597177,-5.930825,"{'fsq_id': '5ae825e4ad910e002c72a002', 'catego...","{'fsq_id': '4b64a08bf964a52082c32ae3', 'catego...","{'fsq_id': '4fc21ef5e4b099ce175faf24', 'catego...","{'fsq_id': '4bb3c394715eef3b6bce86bb', 'catego...","{'fsq_id': '5a37b9bd16fa04126cd4ccd7', 'catego...","{'fsq_id': '4bd352d741b9ef3b0fe3ffe5', 'catego...","{'fsq_id': '4c4647bd342c1b8d3040b689', 'catego...","{'fsq_id': '4b459b93f964a5207d0e26e3', 'catego...","{'fsq_id': '4b071106f964a52093f622e3', 'catego...","{'fsq_id': '5a1fedbcbcbf7a12b589f1f9', 'catego...",2
2,54.588123,-5.935525,"{'fsq_id': '545b434b498e9a85bdb35967', 'catego...","{'fsq_id': '56f3e2b5498e6346b7f91b31', 'catego...","{'fsq_id': '4c39a7270a71c9b6e7d642c9', 'catego...","{'fsq_id': '4b7554a1f964a520ac062ee3', 'catego...","{'fsq_id': '4c191a0c4ff90f475e6b0f49', 'catego...","{'fsq_id': '4b459b93f964a5207d0e26e3', 'catego...","{'fsq_id': '4b7d734ff964a52022bf2fe3', 'catego...","{'fsq_id': '4bf0422df831c92849cd01f2', 'catego...","{'fsq_id': '504b9545e4b0b18004d43314', 'catego...","{'fsq_id': '4b65d319f964a5203a022be3', 'catego...",3
3,54.589329,-5.933508,"{'fsq_id': '545b434b498e9a85bdb35967', 'catego...","{'fsq_id': '4c39a7270a71c9b6e7d642c9', 'catego...","{'fsq_id': '4b459b93f964a5207d0e26e3', 'catego...","{'fsq_id': '4b7554a1f964a520ac062ee3', 'catego...","{'fsq_id': '56f3e2b5498e6346b7f91b31', 'catego...","{'fsq_id': '504b9545e4b0b18004d43314', 'catego...","{'fsq_id': '4b65d319f964a5203a022be3', 'catego...","{'fsq_id': '4bacb913f964a520e8063be3', 'catego...","{'fsq_id': '4c191a0c4ff90f475e6b0f49', 'catego...","{'fsq_id': '4c4647bd342c1b8d3040b689', 'catego...",4
4,54.594821,-5.933116,"{'fsq_id': '4c4647bd342c1b8d3040b689', 'catego...","{'fsq_id': '4b459b93f964a5207d0e26e3', 'catego...","{'fsq_id': '4bdabe62c79cc928a48b7fe9', 'catego...","{'fsq_id': '4b50a28ff964a520762a27e3', 'catego...","{'fsq_id': '4bacb913f964a520e8063be3', 'catego...","{'fsq_id': '4b6031acf964a52060d929e3', 'catego...","{'fsq_id': '4fc21ef5e4b099ce175faf24', 'catego...","{'fsq_id': '57ac99ec498e2258ef0521d3', 'catego...","{'fsq_id': '5ae825e4ad910e002c72a002', 'catego...","{'fsq_id': '4b64a08bf964a52082c32ae3', 'catego...",5


When I tried to do this inside a function, all of the values got shuffled around somehow, and I had a pub that was a museum and also a park, and a sushi restaurant, which it absolutely was NOT when I went and gave myself a headache reading the raw code. So I am just going to do it by hand, where I can check that it isn't doing that. Which sucks but I can't lose another whole day to this. This way I can see that the entries are staying in the correct order.

In [566]:
expanded0 = pd.json_normalize(cleaning[0])
expanded0.head(5)

,fsq_id,categories,chains,closed_bucket,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,...,location.post_town,location.postcode,location.region,related_places.children,geocodes.drop_off.latitude,geocodes.drop_off.longitude,related_places.parent.fsq_id,related_places.parent.categories,related_places.parent.name,location.address_extended
0,50ca5592e4b0cc5bad7c6449,"[{'id': 13003, 'name': 'Bar', 'short_name': 'B...",[],Unsure,557,/v3/places/50ca5592e4b0cc5bad7c6449,The Sunflower,Europe/London,54.603042,-5.932791,...,Belfast,BT1 2JG,Belfast,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ae825e4ad910e002c72a002,"[{'id': 13032, 'name': 'Cafe, Coffee, and Tea ...",[{'id': 'd5719cc0-d890-0132-61d3-7a163eb2a6fc'...,VeryLikelyOpen,72,/v3/places/5ae825e4ad910e002c72a002,Tim Hortons,Europe/London,54.597338,-5.931993,...,Belfast,BT1 5EX,Belfast,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,545b434b498e9a85bdb35967,"[{'id': 13006, 'name': 'Beer Bar', 'short_name...",[],VeryLikelyOpen,107,/v3/places/545b434b498e9a85bdb35967,The Woodworkers,Europe/London,54.588852,-5.934417,...,Belfast,BT7 1RS,Belfast,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,545b434b498e9a85bdb35967,"[{'id': 13006, 'name': 'Beer Bar', 'short_name...",[],VeryLikelyOpen,83,/v3/places/545b434b498e9a85bdb35967,The Woodworkers,Europe/London,54.588852,-5.934417,...,Belfast,BT7 1RS,Belfast,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4c4647bd342c1b8d3040b689,"[{'id': 13018, 'name': 'Pub', 'short_name': 'P...",[],Unsure,75,/v3/places/4c4647bd342c1b8d3040b689,The Crown Liquor Saloon,Europe/London,54.594741,-5.934222,...,Belfast,BT2 7BA,NaN,"[{'fsq_id': '4f71e404e4b072f6771014ac', 'categ...",NaN,NaN,NaN,NaN,NaN,NaN


In [567]:
cats0 = pd.json_normalize(expanded0['categories'])
cats0.head(5)

,0,1,2
0,"{'id': 13003, 'name': 'Bar', 'short_name': 'Ba...","{'id': 13145, 'name': 'Fast Food Restaurant', ...",None
1,"{'id': 13032, 'name': 'Cafe, Coffee, and Tea H...","{'id': 13043, 'name': 'Donut Shop', 'short_nam...","{'id': 13065, 'name': 'Restaurant', 'short_nam..."
2,"{'id': 13006, 'name': 'Beer Bar', 'short_name'...","{'id': 13010, 'name': 'Dive Bar', 'short_name'...","{'id': 13065, 'name': 'Restaurant', 'short_nam..."
3,"{'id': 13006, 'name': 'Beer Bar', 'short_name'...","{'id': 13010, 'name': 'Dive Bar', 'short_name'...","{'id': 13065, 'name': 'Restaurant', 'short_nam..."
4,"{'id': 13018, 'name': 'Pub', 'short_name': 'Pu...","{'id': 13145, 'name': 'Fast Food Restaurant', ...",None


In [568]:
cats0_0 = pd.json_normalize(cats0[0])
cats0_0.head(5)

,id,name,short_name,plural_name,icon.prefix,icon.suffix
0,13003,Bar,Bar,Bars,https://ss3.4sqi.net/img/categories_v2/nightli...,.png
1,13032,"Cafe, Coffee, and Tea House","Cafe, Coffee, and Tea House","Cafes, Coffee, and Tea Houses",https://ss3.4sqi.net/img/categories_v2/food/co...,.png
2,13006,Beer Bar,Beer Bar,Beer Bars,https://ss3.4sqi.net/img/categories_v2/nightli...,.png
3,13006,Beer Bar,Beer Bar,Beer Bars,https://ss3.4sqi.net/img/categories_v2/nightli...,.png
4,13018,Pub,Pub,Pubs,https://ss3.4sqi.net/img/categories_v2/nightli...,.png


In [569]:
cats0_1 = pd.json_normalize(cats0[1])
cats0_2 = pd.json_normalize(cats0[2])

In [570]:
expanded0['id1'] = cats0_0['id']
expanded0['name1'] = cats0_0['name']
expanded0['id2'] = cats0_1['id']
expanded0['name2'] = cats0_1['name']
expanded0['id3'] = cats0_2['id']
expanded0['name3'] = cats0_2['name']
expanded0['categorized'] = 'temp'
expanded0['station_latitude'] = cleaning['station_latitude']
expanded0['station_longitude'] = cleaning['station_longitude']
expanded0['station_num'] = cleaning['station_num']

In [571]:
expanded1 = pd.json_normalize(cleaning[1])
cats1 = pd.json_normalize(expanded1['categories'])
cats1.head(1)

,0,1,2
0,"{'id': 10027, 'name': 'Museum', 'short_name': ...",None,None


In [572]:
cats1_0 = pd.json_normalize(cats1[0])
cats1_1 = pd.json_normalize(cats1[1])
cats1_2 = pd.json_normalize(cats1[2])

In [573]:
expanded1['id1'] = cats1_0['id']
expanded1['name1'] = cats1_0['name']
expanded1['id2'] = cats1_1['id']
expanded1['name2'] = cats1_1['name']
expanded1['id3'] = cats1_2['id']
expanded1['name3'] = cats1_2['name']
expanded1['categorized'] = 'temp'
expanded1['station_latitude'] = cleaning['station_latitude']
expanded1['station_longitude'] = cleaning['station_longitude']
expanded1['station_num'] = cleaning['station_num']

In [574]:
expanded2 = pd.json_normalize(cleaning[2])
cats2 = pd.json_normalize(expanded2['categories'])
cats2.head(1)

,0,1,2,3,4
0,"{'id': 13035, 'name': 'Coffee Shop', 'short_na...","{'id': 13065, 'name': 'Restaurant', 'short_nam...",None,None,None


In [575]:
cats2_0 = pd.json_normalize(cats2[0])
cats2_1 = pd.json_normalize(cats2[1])
cats2_2 = pd.json_normalize(cats2[2])
cats2_3 = pd.json_normalize(cats2[3])
cats2_4 = pd.json_normalize(cats2[4])

In [576]:
expanded2['id1'] = cats2_0['id']
expanded2['name1'] = cats2_0['name']
expanded2['id2'] = cats2_1['id']
expanded2['name2'] = cats2_1['name']
expanded2['id3'] = cats2_2['id']
expanded2['name3'] = cats2_2['name']
expanded2['id4'] = cats2_3['id']
expanded2['name4'] = cats2_3['name']
expanded2['id5'] = cats2_4['id']
expanded2['name5'] = cats2_4['name']
expanded2['categorized'] = 'temp'
expanded2['station_latitude'] = cleaning['station_latitude']
expanded2['station_longitude'] = cleaning['station_longitude']
expanded2['station_num'] = cleaning['station_num']

In [577]:
expanded3 = pd.json_normalize(cleaning[3])
cats3 = pd.json_normalize(expanded3['categories'])
cats3.head(1)

,0,1,2
0,"{'id': 13018, 'name': 'Pub', 'short_name': 'Pu...",None,None


In [578]:
cats3_0 = pd.json_normalize(cats3[0])
cats3_1 = pd.json_normalize(cats3[1])
cats3_2 = pd.json_normalize(cats3[2])

In [579]:
expanded3['id1'] = cats3_0['id']
expanded3['name1'] = cats3_0['name']
expanded3['id2'] = cats3_1['id']
expanded3['name2'] = cats3_1['name']
expanded3['id3'] = cats3_2['id']
expanded3['name3'] = cats3_2['name']
expanded3['categorized'] = 'temp'
expanded3['station_latitude'] = cleaning['station_latitude']
expanded3['station_longitude'] = cleaning['station_longitude']
expanded3['station_num'] = cleaning['station_num']

In [580]:
expanded4 = pd.json_normalize(cleaning[4])
cats4 = pd.json_normalize(expanded4['categories'])
cats4.head(1)

,0,1,2,3,4,5,6
0,"{'id': 13031, 'name': 'Burger Joint', 'short_n...","{'id': 13068, 'name': 'American Restaurant', '...",None,None,None,None,None


In [581]:
cats4_0 = pd.json_normalize(cats4[0])
cats4_1 = pd.json_normalize(cats4[1])
cats4_2 = pd.json_normalize(cats4[2])
cats4_3 = pd.json_normalize(cats4[3])
cats4_4 = pd.json_normalize(cats4[4])
cats4_5 = pd.json_normalize(cats4[5])
cats4_6 = pd.json_normalize(cats4[6])

In [582]:
expanded4['id1'] = cats4_0['id']
expanded4['name1'] = cats4_0['name']
expanded4['id2'] = cats4_1['id']
expanded4['name2'] = cats4_1['name']
expanded4['id3'] = cats4_2['id']
expanded4['name3'] = cats4_2['name']
expanded4['id4'] = cats4_3['id']
expanded4['name4'] = cats4_3['name']
expanded4['id5'] = cats4_4['id']
expanded4['name5'] = cats4_4['name']
expanded4['id6'] = cats4_5['id']
expanded4['name6'] = cats4_5['name']
expanded4['id7'] = cats4_6['id']
expanded4['name7'] = cats4_6['name']
expanded4['categorized'] = 'temp'
expanded4['station_latitude'] = cleaning['station_latitude']
expanded4['station_longitude'] = cleaning['station_longitude']
expanded4['station_num'] = cleaning['station_num']

In [583]:
expanded5 = pd.json_normalize(cleaning[5])
cats5 = pd.json_normalize(expanded5['categories'])
cats5.head(1)

,0,1,2,3,4,5,6
0,"{'id': 13006, 'name': 'Beer Bar', 'short_name'...","{'id': 13145, 'name': 'Fast Food Restaurant', ...",None,None,None,None,None


In [584]:
cats5_0 = pd.json_normalize(cats5[0])
cats5_1 = pd.json_normalize(cats5[1])
cats5_2 = pd.json_normalize(cats5[2])
cats5_3 = pd.json_normalize(cats5[3])
cats5_4 = pd.json_normalize(cats5[4])
cats5_5 = pd.json_normalize(cats5[5])
cats5_6 = pd.json_normalize(cats5[6])

In [585]:
expanded5['id1'] = cats5_0['id']
expanded5['name1'] = cats5_0['name']
expanded5['id2'] = cats5_1['id']
expanded5['name2'] = cats5_1['name']
expanded5['id3'] = cats5_2['id']
expanded5['name3'] = cats5_2['name']
expanded5['id4'] = cats5_3['id']
expanded5['name4'] = cats5_3['name']
expanded5['id5'] = cats5_4['id']
expanded5['name5'] = cats5_4['name']
expanded5['id6'] = cats5_5['id']
expanded5['name6'] = cats5_5['name']
expanded5['id7'] = cats5_6['id']
expanded5['name7'] = cats5_6['name']
expanded5['categorized'] = 'temp'
expanded5['station_latitude'] = cleaning['station_latitude']
expanded5['station_longitude'] = cleaning['station_longitude']
expanded5['station_num'] = cleaning['station_num']

In [586]:
expanded6 = pd.json_normalize(cleaning[6])
cats6 = pd.json_normalize(expanded6['categories'])
cats6.head(1)

,0,1,2
0,"{'id': 13389, 'name': 'Irish Pub', 'short_name...",None,None


In [587]:
cats6_0 = pd.json_normalize(cats6[0])
cats6_1 = pd.json_normalize(cats6[1])
cats6_2 = pd.json_normalize(cats6[2])

In [588]:
expanded6['id1'] = cats6_0['id']
expanded6['name1'] = cats6_0['name']
expanded6['id2'] = cats6_1['id']
expanded6['name2'] = cats6_1['name']
expanded6['id3'] = cats6_2['id']
expanded6['name3'] = cats6_2['name']
expanded6['categorized'] = 'temp'
expanded6['station_latitude'] = cleaning['station_latitude']
expanded6['station_longitude'] = cleaning['station_longitude']
expanded6['station_num'] = cleaning['station_num']

In [589]:
expanded7 = pd.json_normalize(cleaning[7])
cats7 = pd.json_normalize(expanded7['categories'])
cats7.head(1)

,0,1,2
0,"{'id': 13018, 'name': 'Pub', 'short_name': 'Pu...","{'id': 13065, 'name': 'Restaurant', 'short_nam...",None


In [590]:
cats7_0 = pd.json_normalize(cats7[0])
cats7_1 = pd.json_normalize(cats7[1])
cats7_2 = pd.json_normalize(cats7[2])

In [591]:
expanded7['id1'] = cats7_0['id']
expanded7['name1'] = cats7_0['name']
expanded7['id2'] = cats7_1['id']
expanded7['name2'] = cats7_1['name']
expanded7['id3'] = cats7_2['id']
expanded7['name3'] = cats7_2['name']
expanded7['categorized'] = 'temp'
expanded7['station_latitude'] = cleaning['station_latitude']
expanded7['station_longitude'] = cleaning['station_longitude']
expanded7['station_num'] = cleaning['station_num']

In [592]:
expanded8 = pd.json_normalize(cleaning[8])
cats8 = pd.json_normalize(expanded8['categories'])
cats8.head(1)

,0,1,2
0,"{'id': 13013, 'name': 'Hotel Bar', 'short_name...","{'id': 13018, 'name': 'Pub', 'short_name': 'Pu...","{'id': 13065, 'name': 'Restaurant', 'short_nam..."


In [593]:
cats8_0 = pd.json_normalize(cats8[0])
cats8_1 = pd.json_normalize(cats8[1])
cats8_2 = pd.json_normalize(cats8[2])

In [594]:
expanded8['id1'] = cats8_0['id']
expanded8['name1'] = cats8_0['name']
expanded8['id2'] = cats8_1['id']
expanded8['name2'] = cats8_1['name']
expanded8['id3'] = cats8_2['id']
expanded8['name3'] = cats8_2['name']
expanded8['categorized'] = 'temp'
expanded8['station_latitude'] = cleaning['station_latitude']
expanded8['station_longitude'] = cleaning['station_longitude']
expanded8['station_num'] = cleaning['station_num']

In [595]:
expanded9 = pd.json_normalize(cleaning[9])
cats9 = pd.json_normalize(expanded9['categories'])
cats9.head(1)

,0,1,2
0,"{'id': 10037, 'name': 'Concert Hall', 'short_n...","{'id': 10039, 'name': 'Music Venue', 'short_na...","{'id': 13003, 'name': 'Bar', 'short_name': 'Ba..."


In [596]:
cats9_0 = pd.json_normalize(cats9[0])
cats9_1 = pd.json_normalize(cats9[1])
cats9_2 = pd.json_normalize(cats9[2])

In [597]:
expanded9['id1'] = cats9_0['id']
expanded9['name1'] = cats9_0['name']
expanded9['id2'] = cats9_1['id']
expanded9['name2'] = cats9_1['name']
expanded9['id3'] = cats9_2['id']
expanded9['name3'] = cats9_2['name']
expanded9['categorized'] = 'temp'
expanded9['station_latitude'] = cleaning['station_latitude']
expanded9['station_longitude'] = cleaning['station_longitude']
expanded9['station_num'] = cleaning['station_num']

Tossing the mostly empty, pointless, and/or reduntant columns-

In [598]:
expanded0 = expanded0.drop(['categories','chains','link','timezone','geocodes.main.latitude','geocodes.main.longitude','geocodes.roof.latitude',
    'geocodes.roof.longitude','location.admin_region','location.country','location.cross_street','location.formatted_address','location.locality',
    'location.post_town','location.postcode','location.region','related_places.children','geocodes.drop_off.latitude','geocodes.drop_off.longitude',
    'related_places.parent.fsq_id','related_places.parent.categories','related_places.parent.name','location.address_extended'],axis=1)

In [599]:
expanded1=expanded1.drop(['categories','chains','link','timezone','geocodes.main.latitude','geocodes.main.longitude','geocodes.roof.latitude',
    'geocodes.roof.longitude','location.admin_region','location.country','location.cross_street','location.formatted_address','location.locality',
    'location.post_town','location.postcode','location.region','related_places.children','geocodes.drop_off.latitude','geocodes.drop_off.longitude',
    'related_places.parent.fsq_id','related_places.parent.categories','related_places.parent.name','location.address_extended'],axis=1)

In [600]:
expanded2=expanded2.drop(['categories','chains','link','timezone','geocodes.main.latitude','geocodes.main.longitude','geocodes.roof.latitude',
    'geocodes.roof.longitude','location.admin_region','location.country','location.cross_street','location.formatted_address','location.locality',
    'location.post_town','location.postcode','location.region','related_places.children','geocodes.drop_off.latitude','geocodes.drop_off.longitude',
    'related_places.parent.fsq_id','related_places.parent.categories','related_places.parent.name','location.address_extended'],axis=1)

In [601]:
expanded3=expanded3.drop(['categories','chains','link','timezone','geocodes.main.latitude','geocodes.main.longitude','geocodes.roof.latitude',
    'geocodes.roof.longitude','location.admin_region','location.country','location.cross_street','location.formatted_address','location.locality',
    'location.post_town','location.postcode','location.region','related_places.children','geocodes.drop_off.latitude','geocodes.drop_off.longitude',
    'related_places.parent.fsq_id','related_places.parent.categories','related_places.parent.name','location.address_extended'],axis=1)

In [602]:
expanded4=expanded4.drop(['categories','chains','link','timezone','geocodes.main.latitude','geocodes.main.longitude','geocodes.roof.latitude',
    'geocodes.roof.longitude','location.admin_region','location.country','location.cross_street','location.formatted_address','location.locality',
    'location.post_town','location.postcode','location.region','related_places.children','geocodes.drop_off.latitude','geocodes.drop_off.longitude',
    'related_places.parent.fsq_id','related_places.parent.categories','related_places.parent.name','location.address_extended'],axis=1)

In [603]:
expanded5=expanded5.drop(['categories','chains','link','timezone','geocodes.main.latitude','geocodes.main.longitude','geocodes.roof.latitude',
    'geocodes.roof.longitude','location.admin_region','location.country','location.cross_street','location.formatted_address','location.locality',
    'location.post_town','location.postcode','location.region','related_places.children','geocodes.drop_off.latitude','geocodes.drop_off.longitude',
    'related_places.parent.fsq_id','related_places.parent.categories','related_places.parent.name','location.address_extended'],axis=1)

In [604]:
expanded6=expanded6.drop(['categories','chains','link','timezone','geocodes.main.latitude','geocodes.main.longitude','geocodes.roof.latitude',
    'geocodes.roof.longitude','location.admin_region','location.country','location.cross_street','location.formatted_address','location.locality',
    'location.post_town','location.postcode','location.region','related_places.children','geocodes.drop_off.latitude','geocodes.drop_off.longitude',
    'related_places.parent.fsq_id','related_places.parent.categories','related_places.parent.name','location.address_extended'],axis=1)

In [605]:
expanded7=expanded7.drop(['categories','chains','link','timezone','geocodes.main.latitude','geocodes.main.longitude','geocodes.roof.latitude',
    'geocodes.roof.longitude','location.admin_region','location.country','location.cross_street','location.formatted_address','location.locality',
    'location.post_town','location.postcode','location.region','related_places.children','geocodes.drop_off.latitude','geocodes.drop_off.longitude',
    'related_places.parent.fsq_id','related_places.parent.categories','related_places.parent.name','location.address_extended'],axis=1)

In [606]:
expanded8=expanded8.drop(['categories','chains','link','timezone','geocodes.main.latitude','geocodes.main.longitude','geocodes.roof.latitude',
    'geocodes.roof.longitude','location.admin_region','location.country','location.cross_street','location.formatted_address','location.locality',
    'location.post_town','location.postcode','location.region','related_places.children','geocodes.drop_off.latitude','geocodes.drop_off.longitude',
    'related_places.parent.fsq_id','related_places.parent.categories','related_places.parent.name','location.address_extended'],axis=1)

In [607]:
expanded9=expanded9.drop(['categories','chains','link','timezone','geocodes.main.latitude','geocodes.main.longitude','geocodes.roof.latitude',
    'geocodes.roof.longitude','location.admin_region','location.country','location.cross_street','location.formatted_address','location.locality',
    'location.post_town','location.postcode','location.region','related_places.children','geocodes.drop_off.latitude','geocodes.drop_off.longitude',
    'related_places.parent.fsq_id','related_places.parent.categories','related_places.parent.name','location.address_extended'],axis=1)

In [608]:
expanded8[expanded8['name1']=='Burger Joint']

,fsq_id,closed_bucket,distance,name,location.address,id1,name1,id2,name2,id3,name3,categorized,station_latitude,station_longitude,station_num
13,59cf9dc2aa6c9536b0f78460,VeryLikelyOpen,445,Tribal Burger,86 Botanic Ave,13031,Burger Joint,NaN,NaN,NaN,NaN,temp,54.586848,-5.939333,14
23,59cf9dc2aa6c9536b0f78460,VeryLikelyOpen,446,Tribal Burger,86 Botanic Ave,13031,Burger Joint,NaN,NaN,NaN,NaN,temp,54.583948,-5.936989,24
26,59cf9dc2aa6c9536b0f78460,VeryLikelyOpen,341,Tribal Burger,86 Botanic Ave,13031,Burger Joint,NaN,NaN,NaN,NaN,temp,54.583901,-5.932810,27
30,5a8ebe53a9fe123e9ff58e3b,VeryLikelyOpen,821,McDonald's,233 Shore Rd,13031,Burger Joint,13145.0,Fast Food Restaurant,NaN,NaN,temp,54.621307,-5.923744,31


Aaaaaand now to go put a final tag on these. <br>
I'm going to stick with my original 'if any of the columns say bar, its a bar' policy. Last time I had a pub that, even ignoring the nonsense where it was somehow a museum and a park, had a random mix of other tags, so lets see if they come out consistent this time...  Cafés and coffe shops are just going to be 'coffee shops'. After much consideration I have decided bakeries are also going under the 'coffee shop' umbrella.

In [609]:
parks0 = [7,26,28,30,49,56]
restaurant0= [24,34,40,48,53,54,13,17,18,38,51,4,9,15,25,32,8,19,11,22,41,36,46,47,55,34,39,31]
bar0 = [0,10,21,42,2,3,5,37,20,27,29]
museum0 = [33,35,52,14,23]
coffee0 = [6,16,45,50,1,44]
stadiums0 = [12,43]

In [610]:
expanded0.loc[stadiums0,'categorized'] = 'Stadium'
expanded0.loc[parks0,'categorized'] = 'Park'
expanded0.loc[restaurant0,'categorized'] = 'Restaurant'
expanded0.loc[bar0,'categorized'] = 'Bar'
expanded0.loc[museum0,'categorized'] = 'Museum'
expanded0.loc[coffee0,'categorized'] = 'Coffee Shop'

So far no stupidity...

In [611]:
museums1=[6,16,26,45,49,56,0,50]
restaurant1=[18,20,21,27,29,43,1,19,42,51,54,2,4,23,28,46,5,37,7,12,41,36,44,48,39,17,25,15,3,55]
bar1=[9,32,35,52,24,40,53,13,22,34,47,33,31]
park1=[11,38,8]
library1=[30]
coffee1=[10,14]

In [612]:
expanded1.loc[park1,'categorized'] = 'Park'
expanded1.loc[restaurant1,'categorized'] = 'Restaurant'
expanded1.loc[bar1,'categorized'] = 'Bar'
expanded1.loc[museums1,'categorized'] = 'Museum'
expanded1.loc[coffee1,'categorized'] = 'Coffee Shop'
expanded1.loc[library1,'categorized'] = 'Library'

In [613]:
restaurants2=[1,5,9,19,20,33,47,50,3,26,27,37,44,49,10,15,41,4,18,51,21,12,2,54,11,30,13,55,40,36,43]
bars2=[28,25,29,34,48,24,52,53,31]
coffee2=[0,17,22,32,38,35,46,56,7,39]
museums2=[6,16,45,42]
parks2=[23]
stadium2=[8]
monument2=[14]

In [614]:
expanded2.loc[restaurants2,'categorized'] = 'Restaurant'
expanded2.loc[bars2,'categorized'] = 'Bar'
expanded2.loc[coffee2,'categorized'] = 'Coffee Shop'
expanded2.loc[museums2,'categorized'] = 'Museum'
expanded2.loc[parks2,'categorized'] = 'Park'
expanded2.loc[stadium2,'categorized'] = 'Stadium'
expanded2.loc[monument2,'categorized'] = 'Monument'

In [615]:
restaurants3=[25,31,37,41,51,12,21,32,53,0,10,27,5,44,47,24,29,8,49,7,33,2,3,55,52,43,36,17,30,26,20,56,9]
bars3=[1,40,15,22,42,48,35,39,18,34,4,19,28]
coffee3=[23,46,50,54]
museums3=[13,14,38,6,16,45]
hospital3=[11]

In [616]:
expanded3.loc[restaurants3,'categorized'] = 'Restaurant'
expanded3.loc[bars3,'categorized'] = 'Bar'
expanded3.loc[coffee3,'categorized'] = 'Coffee Shop'
expanded3.loc[museums3,'categorized'] = 'Museum'
expanded3.loc[hospital3,'categorized'] = 'Hospital'

In [617]:
restaurants4=[0,24,43,50,5,36,37,47,42,54,34,53,1,4,18,29,21,44,3,8,2,23,33,13,19,38,30,52,35,32]
bars4=[10,39,9,20,22,31,51]
coffee4=[28,41,49,55,15,46,48,27,7]
museums4=[14]
parks4=[11,17]
monument4=[6,16,45,12,25]
bot_garden4=[26,56]
mall4=[40]

In [618]:
expanded4.loc[restaurants4,'categorized'] = 'Restaurant'
expanded4.loc[bars4,'categorized'] = 'Bar'
expanded4.loc[coffee4,'categorized'] = 'Coffee Shop'
expanded4.loc[museums4,'categorized'] = 'Museum'
expanded4.loc[parks4,'categorized'] = 'Park'
expanded4.loc[monument4,'categorized'] = 'Monument'
expanded4.loc[bot_garden4,'categorized'] = 'Botanical Garden'
expanded4.loc[mall4,'categorized'] = 'Mall'

In [619]:
restaurants5=[3,7,11,28,30,41,18,34,12,35,46,56,20,33,52,1,10,29,21,40,31,2,54,25,27,47,55]
bars5=[42,50,0,23,32,5,15,37,53,22,48]
coffee5=[4,26,43,51,8,44,9]
museums5=[6,16,45,14]
parks5=[13]
monument5=[19,36]
bot_garden5=[38,49]
mall5=[24]
stadium5=[17]
tourism5=[39]

In [620]:
expanded5.loc[restaurants5,'categorized'] = 'Restaurant'
expanded5.loc[bars5,'categorized'] = 'Bar'
expanded5.loc[coffee5,'categorized'] = 'Coffee Shop'
expanded5.loc[museums5,'categorized'] = 'Museum'
expanded5.loc[parks5,'categorized'] = 'Bar'
expanded5.loc[monument5,'categorized'] = 'Monument'
expanded5.loc[bot_garden5,'categorized'] = 'Botanical Garden'
expanded5.loc[mall5,'categorized'] = 'Mall'
expanded5.loc[stadium5,'categorized'] = 'Stadium'
expanded5.loc[tourism5,'categorized'] = 'Tourism'

In [621]:
restaurants6=[1,5,10,27,38,47,4,16,34,36,9,30,31,51,56,46,39,44,13,17,19,26,25,54,21,29,7,14,0,22,24,40]
bars6=[11,33,32,3,48,53,15,50]
coffee6=[12,18,43,8,49,52]
parks6=[2,37]
monument6=[42]
bot_garden6=[23]
mall6=[20]
stadium6=[55,6,45]
hospital6=[35]
city_hall6=[41]
tourism6=[28]

In [622]:
expanded6.loc[restaurants6,'categorized'] = 'Restaurant'
expanded6.loc[bars6,'categorized'] = 'Bar'
expanded6.loc[coffee6,'categorized'] = 'Coffee Shop'
expanded6.loc[parks6,'categorized'] = 'Bar'
expanded6.loc[monument6,'categorized'] = 'Monument'
expanded6.loc[bot_garden6,'categorized'] = 'Botanical Garden'
expanded6.loc[mall6,'categorized'] = 'Mall'
expanded6.loc[stadium6,'categorized'] = 'Stadium'
expanded6.loc[hospital6,'categorized'] = 'Hospital'
expanded6.loc[city_hall6,'categorized'] = 'City Hall'
expanded6.loc[tourism6,'categorized'] = 'Tourism'

In [623]:
restaurants7=[3,5,22,29,37,39,53,0,32,42,1,13,46,27,51,6,45,4,23,30,11,8,49,15,48,7,19,55,28,38,9,20,54]
bars7=[24,26,40,10,25,24,35,21,18,34]
coffee7=[43,47,56,12,33,36,31,41]
parks7=[17]
tourism7=[44]
museum7=[2,14,50]
library7=[52]
stadium7=[16]

In [624]:
expanded7.loc[restaurants7,'categorized'] = 'Restaurant'
expanded7.loc[bars7,'categorized'] = 'Bar'
expanded7.loc[coffee7,'categorized'] = 'Coffee Shop'
expanded7.loc[parks7,'categorized'] = 'Park'
expanded7.loc[tourism7,'categorized'] = 'Tourism'
expanded7.loc[museum7,'categorized'] = 'Museum'
expanded7.loc[library7,'categorized'] = 'Library'
expanded7.loc[stadium7,'categorized'] = 'Stadium'

In [625]:
restaurants8=[13,23,26,30,17,18,19,47,5,12,21,50,20,6,27,41,2,7,49,51,52,31,25,3,38,22,48]
bars8=[15,28,33,35,36,43,40,37,44,53,0,10,32,1,54]
coffee8=[8,9,14,24,39,46,42,55,56,4]
monument8=[16]
mall8=[29,34]
concert8=[45]
library8=[11]


In [626]:
expanded8.loc[restaurants8,'categorized'] = 'Restaurant'
expanded8.loc[bars8,'categorized'] = 'Bar'
expanded8.loc[coffee8,'categorized'] = 'Coffee Shop'
expanded8.loc[monument8,'categorized'] = 'Monument'
expanded8.loc[mall8,'categorized'] = 'Mall'
expanded8.loc[concert8,'categorized'] = 'Concert Hall'
expanded8.loc[library8,'categorized'] = 'Library'

In [627]:
restaurants9=[3,4,8,9,16,20,24,25,32,37,49,12,18,27,31,40,47,51,26,19,30,7,28,43,1,55,53,46,36,41,11,23,35]
bars9=[45,10,34,17,38,33,54,42,2]
coffee9=[5,13,14,56,39,44,15,29]
parks9=[21]
monument9=[48]
hospital9=[52]
city_hall9=[6]
stadium9=[50]
concert9=[0,22]

In [628]:
expanded9.loc[restaurants9,'categorized'] = 'Restaurant'
expanded9.loc[bars9,'categorized'] = 'Bar'
expanded9.loc[coffee9,'categorized'] = 'Coffee Shop'
expanded9.loc[parks9,'categorized'] = 'Park'
expanded9.loc[monument9,'categorized'] = 'Monument'
expanded9.loc[hospital9,'categorized'] = 'Hospital'
expanded9.loc[city_hall9,'categorized'] = 'City Hall'
expanded9.loc[stadium9,'categorized'] = 'Stadium'
expanded9.loc[concert9,'categorized'] = 'Concert Hall'

In [629]:
print(expanded0[expanded0['categorized']=='temp'].shape)
print(expanded1[expanded1['categorized']=='temp'].shape)
print(expanded2[expanded2['categorized']=='temp'].shape)
print(expanded3[expanded3['categorized']=='temp'].shape)
print(expanded4[expanded4['categorized']=='temp'].shape)
print(expanded5[expanded5['categorized']=='temp'].shape)
print(expanded6[expanded6['categorized']=='temp'].shape)
print(expanded7[expanded7['categorized']=='temp'].shape)
print(expanded8[expanded8['categorized']=='temp'].shape)
print(expanded9[expanded9['categorized']=='temp'].shape)

(0, 15)
(0, 15)
(0, 21)
(0, 15)
(0, 23)
(0, 23)
(0, 15)
(0, 17)
(0, 15)
(0, 15)


In [630]:
expanded1.shape

(57, 15)

In [631]:
all_cats = pd.concat([expanded0,expanded1,expanded2,expanded3,expanded4,expanded5,expanded6,expanded7,expanded8,expanded9])
all_cats.keys()

Index(['fsq_id', 'closed_bucket', 'distance', 'name', 'location.address',
       'id1', 'name1', 'id2', 'name2', 'id3', 'name3', 'categorized',
       'station_latitude', 'station_longitude', 'station_num',
       'geocodes.front_door.latitude', 'geocodes.front_door.longitude', 'id4',
       'name4', 'id5', 'name5', 'id6', 'name6', 'id7', 'name7'],
      dtype='object')

In [632]:
all_cats = all_cats[['fsq_id','station_num','distance','name','location.address','categorized','closed_bucket']]
all_cats = all_cats.sort_values('station_num').reset_index().drop('index',axis=1)

In [633]:
print(all_cats.isna().value_counts())
print(all_cats['location.address'].isna().value_counts())

fsq_id  station_num  distance  name   location.address  categorized  closed_bucket
False   False        False     False  False             False        False            534
                                      True              False        False             36
Name: count, dtype: int64
location.address
False    534
True      36
Name: count, dtype: int64


In [634]:
points_of_interest = all_cats.fillna('Missing data')

In [635]:
points_of_interest.iloc[31]

fsq_id              4c39a7270a71c9b6e7d642c9
station_num                                4
distance                                  21
name                              Lee Garden
location.address           14-18 Botanic Ave
categorized                       Restaurant
closed_bucket                 VeryLikelyOpen
Name: 31, dtype: object

In [636]:
print(points_of_interest['location.address'].isna().value_counts())

location.address
False    570
Name: count, dtype: int64


In [637]:
points_of_interest['name'].value_counts()

name
Kelly's Cellars    15
Bunsen             14
The Garrick Bar    13
Ox                 13
Whites Tavern      13
                   ..
C View Palace       1
The York            1
Sylvesters Cafe     1
Ardoyne Library     1
Clements Cafe       1
Name: count, Length: 207, dtype: int64

Those top ones are all single locations getting picked up by multiple stations. 

In [638]:
points_of_interest = points_of_interest.rename(columns={'location.address':'location_address'})

**Put your parsed results into a DataFrame**

In [639]:
points_of_interest['categorized'].value_counts()

categorized
Restaurant          303
Bar                 108
Coffee Shop          70
Museum               31
Park                 14
Monument             11
Stadium               9
Mall                  5
Botanical Garden      5
Concert Hall          3
Hospital              3
Library               3
Tourism               3
City Hall             2
Name: count, dtype: int64

In [640]:
points_of_interest[points_of_interest['station_num']==1]

,fsq_id,station_num,distance,name,location_address,categorized,closed_bucket
0,50ca5592e4b0cc5bad7c6449,1,557,The Sunflower,65 Union St,Bar,Unsure
1,520080a3498eb97026f147b8,1,811,Established Coffee,54 Hill St,Coffee Shop,Unsure
2,4b64a08bf964a52082c32ae3,1,920,Kelly's Cellars,30-32 Bank St,Restaurant,Unsure
3,5a1fedbcbcbf7a12b589f1f9,1,952,Bunsen,6-8 Hill St,Restaurant,LikelyOpen
4,4b06f46df964a5201af422e3,1,907,Black Box,18-22 Hill St,Concert Hall,Unsure
5,4bd352d741b9ef3b0fe3ffe5,1,985,Whites Tavern,2-4 Winecellar Entry,Restaurant,Unsure
6,4c1669aea5eb76b040ccc4b7,1,809,The John Hewitt,51 Donegall St,Restaurant,VeryLikelyOpen
7,52988e8511d22448bd236206,1,950,The Dirty Onion,3 Hill St,Bar,Unsure
8,50b0e62be4b0441cd92ec4f3,1,388,Crumlin Road Gaol,53-55 Crumlin Rd,Museum,VeryLikelyOpen
9,4bb897143db7b7135e65219a,1,988,The Spaniard,3 Skipper St,Bar,LikelyOpen


In [641]:
# points_of_interest.to_csv('foursquare_locations.csv')

# Yelp

**Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice.**

In [642]:
load_dotenv()
key_yelp = os.getenv("YELP_KEY")
headers = {"Accept": "application/json", 
           "Authorization" : f"Bearer {key_yelp}" }

In [643]:
poi_cats_list = "bars,food,nightlife,tours,restaurants,museums,stadiumsarenas,parks,gardens,hospitals,libraries,musicvenues,landmarks,townhall,wholesale_stores,deptstores,outlet_stores,cafes"

In [644]:
coordinates = ['latitude=54.607397&longitude=-5.937086','latitude=54.597177&longitude=-5.930825','latitude=54.588123&longitude=-5.935525',
'latitude=54.589329&longitude=-5.933508','latitude=54.594821&longitude=-5.933116','latitude=54.592172&longitude=-5.925547',
'latitude=54.603420&longitude=-5.906825','latitude=54.586112&longitude=-5.891912','latitude=54.572534&longitude=-5.915076',
'latitude=54.599697&longitude=-5.936308','latitude=54.604986&longitude=-5.932853','latitude=54.609674&longitude=-5.972700',
'latitude=54.589851&longitude=-5.958463','latitude=54.586848&longitude=-5.939333','latitude=54.611906&longitude=-5.902814',
'latitude=54.599358&longitude=-5.929838','latitude=54.605713&longitude=-5.911299','latitude=54.578142&longitude=-5.918787',
'latitude=54.592680&longitude=-5.931528','latitude=54.595315&longitude=-5.949091','latitude=54.595255&longitude=-5.923680',
'latitude=54.581962&longitude=-5.977871','latitude=54.601425&longitude=-5.925847','latitude=54.583948&longitude=-5.936989',
'latitude=54.597270&longitude=-5.921116','latitude=54.599399&longitude=-5.947595','latitude=54.583901&longitude=-5.932810',
'latitude=54.595840&longitude=-5.929881','latitude=54.618475&longitude=-5.938534','latitude=54.597573&longitude=-5.927663',
'latitude=54.621307&longitude=-5.923744','latitude=54.590889&longitude=-5.967314','latitude=54.601319&longitude=-5.931308',
'latitude=54.612417&longitude=-5.936748','latitude=54.594321&longitude=-5.915236','latitude=54.608528&longitude=-5.941029',
'latitude=54.598753&longitude=-5.899396','latitude=54.591219&longitude=-5.922650','latitude=54.580556&longitude=-5.920504',
'latitude=54.596086&longitude=-5.892781','latitude=54.598203&longitude=-5.924033','latitude=54.577839&longitude=-5.881805',
'latitude=54.603687&longitude=-5.944403','latitude=54.583056&longitude=-5.956993','latitude=54.598305&longitude=-5.890582',
'latitude=54.600337&longitude=-5.905516','latitude=54.594131&longitude=-5.868545','latitude=54.594634&longitude=-5.927441',
'latitude=54.601437&longitude=-5.921996','latitude=54.574854&longitude=-5.932134','latitude=54.603923&longitude=-5.917243',
'latitude=54.593469&longitude=-5.934409','latitude=54.611849&longitude=-5.942139','latitude=54.595321&longitude=-5.916460',
'latitude=54.597142&longitude=-5.934827','latitude=54.575364&longitude=-5.955373','latitude=54.576116&longitude=-5.942574']

In [645]:
# storage = []
# for string in coordinates:
#     url = 'https://api.yelp.com/v3/businesses/search?' + string + '&categories=' + poi_cats_list +'&radius=1000'
#     result = requests.get(url, headers=headers)
#     storage.append(result.json()) 

In [646]:
# with open('yelp_retrieval.json', 'w') as f:
#     json.dump(storage, f)

**Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)**

Deeply confusing: Yelp says it has a different API endpoint to get ratings, but they still show up on the basic search.

In [647]:
with open('yelp_retrieval.json') as f:
    data=json.load(f)

This seach returned a MUCH chunkier batch of data so I have finally gotten help building a functional unpacking function.

In [648]:
data[0]['businesses'][0].keys()

dict_keys(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count', 'categories', 'rating', 'coordinates', 'transactions', 'price', 'location', 'phone', 'display_phone', 'distance', 'business_hours', 'attributes'])

In [649]:
stationids = []
busID_all = []
bus_name_all = []
bus_cat_all = []
review_count_all = []
rating_all = []
phone_all = []
address_all = []
distance_all = []
bus_hour_all = []

In [650]:
for index, response in enumerate(data,start=1):
    businesses = response['businesses']

    for business in businesses:
        stationids.append(index)

        busID = business['id']
        bus_name = business['name']
        bus_cats = business['categories']
        review_count = business['review_count']
        rating = business['rating']
        phone = business['phone']
        distance = business['distance']
        bus_hour = business['business_hours']

        busID_all.append(busID)
        bus_name_all.append(bus_name)
        bus_cat_all.append(bus_cats)
        review_count_all.append(review_count)
        rating_all.append(rating)
        phone_all.append(phone)
        distance_all.append(distance)
        bus_hour_all.append(bus_hour)

        location = business['location']
        address_all.append(location['address1'])       
        

In [651]:
formatting = ({
    "Station_No": stationids,
    "Business_ID": busID_all,
    "Business_Name": bus_name_all,
    "Category": bus_cat_all,
    "Review_Count": review_count_all,
    "Rating": rating_all,
    "Phone_No": phone_all,
    "Address": address_all,
    "Distance": distance_all,
    "Business_Hours": bus_hour_all})
yelp_locations = pd.DataFrame(formatting)

I'm not going to normalize 'Business Hours', becuase my point is that Yelp has it in detail, where 4Square only had the likelyhood of being open. 

In [652]:
yelp_locations.head(2)

,Station_No,Business_ID,Business_Name,Category,Review_Count,Rating,Phone_No,Address,Distance,Business_Hours
0,1,UAywQCs1e2E3HuFclCEIUQ,Mourne Seafood Bar,"[{'alias': 'seafood', 'title': 'Seafood'}]",120,4.5,+442890248544,34-36 Bank Street,920.674254,"[{'open': [{'is_overnight': False, 'start': '1..."
1,1,5fjrK7NHjRlwND_d3tGdOQ,Coppi,"[{'alias': 'italian', 'title': 'Italian'}]",42,4.4,+442890311959,St Anne's Square,733.445663,"[{'open': [{'is_overnight': False, 'start': '1..."


I really wish these had their parent tags attached... I'm not sorting these into categories, I really don't have time when I have not figured out how to do it with code. Unfortunately. 

In [653]:
yelp_locations[yelp_locations['Business_Name']=="Pizza Express"]

,Station_No,Business_ID,Business_Name,Category,Review_Count,Rating,Phone_No,Address,Distance,Business_Hours
94,5,bJIaVM4QX94mxp8xvL5Evw,Pizza Express,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",27,3.7,+442890329050,25 Bedford Street,159.236414,"[{'open': [{'is_overnight': False, 'start': '1..."
372,19,bJIaVM4QX94mxp8xvL5Evw,Pizza Express,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",27,3.7,+442890329050,25 Bedford Street,132.030072,"[{'open': [{'is_overnight': False, 'start': '1..."
494,25,LSW97kHG9lDwDu4Pqkod-g,Pizza Express,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",17,4.1,+442890314449,Unit TFC6,265.359799,"[{'open': [{'is_overnight': False, 'start': '1..."
554,28,bJIaVM4QX94mxp8xvL5Evw,Pizza Express,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",27,3.7,+442890329050,25 Bedford Street,238.123165,"[{'open': [{'is_overnight': False, 'start': '1..."
679,34,T8QDrRk0jpZHZyf4wsiH4Q,Pizza Express,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2,5.0,+442890319000,6 St Anne's Square,1159.684098,"[{'open': [{'is_overnight': False, 'start': '1..."
818,41,LSW97kHG9lDwDu4Pqkod-g,Pizza Express,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",17,4.1,+442890314449,Unit TFC6,52.373179,"[{'open': [{'is_overnight': False, 'start': '1..."
938,47,X9eMtMUgq0UkQ0vmx4Ey8Q,Pizza Express,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",1,5.0,+442890657772,376-378 Upper Newtownards Road,158.127601,[]
949,48,bJIaVM4QX94mxp8xvL5Evw,Pizza Express,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",27,3.7,+442890329050,25 Bedford Street,262.204615,"[{'open': [{'is_overnight': False, 'start': '1..."
1034,52,bJIaVM4QX94mxp8xvL5Evw,Pizza Express,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",27,3.7,+442890329050,25 Bedford Street,205.876829,"[{'open': [{'is_overnight': False, 'start': '1..."
1049,53,T8QDrRk0jpZHZyf4wsiH4Q,Pizza Express,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2,5.0,+442890319000,6 St Anne's Square,1330.473352,"[{'open': [{'is_overnight': False, 'start': '1..."


**Put your parsed results into a DataFrame**

In [654]:
yelp_locations

,Station_No,Business_ID,Business_Name,Category,Review_Count,Rating,Phone_No,Address,Distance,Business_Hours
0,1,UAywQCs1e2E3HuFclCEIUQ,Mourne Seafood Bar,"[{'alias': 'seafood', 'title': 'Seafood'}]",120,4.5,+442890248544,34-36 Bank Street,920.674254,"[{'open': [{'is_overnight': False, 'start': '1..."
1,1,5fjrK7NHjRlwND_d3tGdOQ,Coppi,"[{'alias': 'italian', 'title': 'Italian'}]",42,4.4,+442890311959,St Anne's Square,733.445663,"[{'open': [{'is_overnight': False, 'start': '1..."
2,1,fXB97OO0E7B-Rvzx5P47uQ,Whites Tavern,"[{'alias': 'irish_pubs', 'title': 'Irish Pub'}...",72,4.1,+442890312582,2-4 Winecellar Entry,982.128952,"[{'open': [{'is_overnight': True, 'start': '11..."
3,1,fJrpJ3cS8Ggy0myftZJyDA,Made in Belfast,"[{'alias': 'british', 'title': 'British'}]",42,4.5,+442895458120,23 Talbot Street,802.068537,[]
4,1,zSdfeTjjz7pIbyYQgeRBiw,Hadskis,"[{'alias': 'modern_european', 'title': 'Modern...",26,4.3,+442890325444,33 Donegal Street,857.123178,"[{'open': [{'is_overnight': False, 'start': '1..."
...,...,...,...,...,...,...,...,...,...,...
1135,57,mL4QJYktBe5W_AwFQhS9VA,Slims Healthy Kitchen,"[{'alias': 'healthmarkets', 'title': 'Health M...",15,3.8,+442895088011,429-431 Lisburn Road,651.218357,"[{'open': [{'is_overnight': False, 'start': '0..."
1136,57,v-ngsUEbix-8-rvAvbdGhg,Welcome Chinese Restaurant,"[{'alias': 'chinese', 'title': 'Chinese'}]",8,4.6,+442890381359,22 Stranmillis Road,705.036003,"[{'open': [{'is_overnight': False, 'start': '1..."
1137,57,dIiIAE1O725S0Wfcg2FQtw,Eddie Rocket's,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",18,3.8,+442890668442,417-419 Lisburn Road,656.307459,"[{'open': [{'is_overnight': False, 'start': '1..."
1138,57,1Z125elooMvcz9BoxCARxw,The Sphinx,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",30,3.8,+442890681881,74 Stranmillis Road,625.845079,"[{'open': [{'is_overnight': False, 'start': '1..."


In [655]:
# yelp_locations.to_csv('yelp_locations.csv')

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Overall, Yelp provides more complete data. Including the tags dropped for redundancy, on a longitude/latitude/radius based query, Foursquare provides:<ul>
- the location's database ID<br>
- the location's categories<br>
- what chain the location is part of, if any<br>
- the likelyhood of the location being open<br>
- the location's name<br>
- the distance from the query coordinates<br>
- the endpoint link for the location<br>
- the location's timezone<br>
- loaction longitude and latitude, in a few configurations: 'main', 'roof' and 'drop off'<br>
- location address, formatted address, cross street and country<br>
- location locality, admin region, post town, postal code and region<br>
- some options for related places; 'children', 'parent fsq id', 'parent categories' and 'parent name'<br>
- extended location address

Many of the later columns are empty in the data I retrieved. The only columns that are actually useful are database ID, categories, distance from coordinates, name, and address. 

Conversly, for a coordinates/radius query, Yelp gives you:<ul>
- the ID<br>
- the location's name
- a link to an image from the location
- the current open/closed status
- a link to the Yelp page itself
- how many people have left reviews, what the rating is
- coordinates and address
- a rating of how expensive it is
- the phone number
- distance from coordinates
- business hours
- info on any temporary closures, if it's open 24 hours, if there's a waitlist, and a link to an online menu if possible</ul>
While I didn't retain _all_ of that for my dataframe, there is significantly more available there than there is on Foursquare. Even just the reviews and price scale is a big difference. 

However, the Yelp API was a lot more annoying to access; you had to put the authorization variable as 'Bearer {key}' and it doesn't tell you that anywhere. I only knew to Google what was going on because my first test was in Postman and it gave me a 'Bearer does not match' output in addition to the 400 Bad Request error. 

Get the top 10 restaurants according to their rating

In [656]:
get_rank = yelp_locations.drop_duplicates('Business_ID')
get_rank.head(2)

,Station_No,Business_ID,Business_Name,Category,Review_Count,Rating,Phone_No,Address,Distance,Business_Hours
0,1,UAywQCs1e2E3HuFclCEIUQ,Mourne Seafood Bar,"[{'alias': 'seafood', 'title': 'Seafood'}]",120,4.5,+442890248544,34-36 Bank Street,920.674254,"[{'open': [{'is_overnight': False, 'start': '1..."
1,1,5fjrK7NHjRlwND_d3tGdOQ,Coppi,"[{'alias': 'italian', 'title': 'Italian'}]",42,4.4,+442890311959,St Anne's Square,733.445663,"[{'open': [{'is_overnight': False, 'start': '1..."


Ok, I now have to sort these, but I'm only going to sort the ones with 5.0 stars, as that is the highest ranking in the data. 

In [657]:
five_star = get_rank[get_rank['Rating']==5.0]
sort = pd.json_normalize(five_star['Category'])
sort['Categorized'] = 'temp'

In [658]:
yelp_restaurants = [5,11,13,14,15,17,19,20,29,30,41,43,44,22,26,39]
yelp_bars = [0,8,40]
yelp_tours = [2,3,9,16,33,35,36,37,38]
yelp_coffee = [1,4,18,24,25,27,32,42,6]
yelp_grocery = [7,10,12,46]
yelp_catering = [45,34]
yelp_monument = [31]
yelp_misc = [21,23,28]

I used the same rules as I did for Foursquare as much as possible, adding the 'Grocery', 'Catering' and 'Misc' categories for various entries with tags that didn't fit any of the other categories. 

In [659]:
sort.loc[yelp_restaurants,'Categorized'] = 'Restaurant'
sort.loc[yelp_bars,'Categorized'] = 'Bar'
sort.loc[yelp_tours,'Categorized'] = 'Tour'
sort.loc[yelp_coffee,'Categorized'] = 'Coffee Shop'
sort.loc[yelp_grocery,'Categorized'] = 'Grocery'
sort.loc[yelp_catering,'Categorized'] = 'Catering'
sort.loc[yelp_monument,'Categorized'] = 'Monument'
sort.loc[yelp_misc,'Categorized'] = 'Misc'

In [660]:
five_star.head()


,Station_No,Business_ID,Business_Name,Category,Review_Count,Rating,Phone_No,Address,Distance,Business_Hours
15,1,xbjmgGMytNipvGpLgy3lYw,The Muddlers Club,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",11,5.0,+442890313199,Unit 1,897.900969,"[{'open': [{'is_overnight': False, 'start': '1..."
133,7,CMAAXOjjRu_gpQNMrVfZxw,Paper Cup,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",3,5.0,+442890766400,13 Block 11 ARC Queens Road,373.373643,"[{'open': [{'is_overnight': False, 'start': '0..."
134,7,yyM6-BakVxcIcblqQph3aQ,The Wee Tram,"[{'alias': 'tours', 'title': 'Tours'}]",3,5.0,,2 Queens Road,319.702024,"[{'open': [{'is_overnight': False, 'start': '1..."
137,7,U5s9ldS6EGHRqn-Y3g6flg,Belfast Bike Tours,"[{'alias': 'biketours', 'title': 'Bike tours'}...",2,5.0,+447733169476,Titanic Quarter,465.838772,"[{'open': [{'is_overnight': False, 'start': '1..."
142,8,kQ8w6myguUmqjHbFFRWHSA,Lazy Claire Patisserie,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",1,5.0,+442890450196,227 Castlereagh Road,74.896650,"[{'open': [{'is_overnight': False, 'start': '1..."


In [663]:
final_cats = list(sort['Categorized'])

In [664]:
five_star_cats = five_star.copy()

In [665]:
five_star_cats['Category'] = final_cats
five_star_cats.head(5)

,Station_No,Business_ID,Business_Name,Category,Review_Count,Rating,Phone_No,Address,Distance,Business_Hours
15,1,xbjmgGMytNipvGpLgy3lYw,The Muddlers Club,Bar,11,5.0,+442890313199,Unit 1,897.900969,"[{'open': [{'is_overnight': False, 'start': '1..."
133,7,CMAAXOjjRu_gpQNMrVfZxw,Paper Cup,Coffee Shop,3,5.0,+442890766400,13 Block 11 ARC Queens Road,373.373643,"[{'open': [{'is_overnight': False, 'start': '0..."
134,7,yyM6-BakVxcIcblqQph3aQ,The Wee Tram,Tour,3,5.0,,2 Queens Road,319.702024,"[{'open': [{'is_overnight': False, 'start': '1..."
137,7,U5s9ldS6EGHRqn-Y3g6flg,Belfast Bike Tours,Tour,2,5.0,+447733169476,Titanic Quarter,465.838772,"[{'open': [{'is_overnight': False, 'start': '1..."
142,8,kQ8w6myguUmqjHbFFRWHSA,Lazy Claire Patisserie,Coffee Shop,1,5.0,+442890450196,227 Castlereagh Road,74.896650,"[{'open': [{'is_overnight': False, 'start': '1..."


In [666]:
five_star_cats[five_star_cats['Category']=='Restaurant'].sort_values('Review_Count',ascending=False)

,Station_No,Business_ID,Business_Name,Category,Review_Count,Rating,Phone_No,Address,Distance,Business_Hours
243,13,lee6C8Nc-7S-fIguMu_q1w,Saphyre Restaurant,Restaurant,7,5.0,+442890688606,135 Lisburn Road,1125.207964,"[{'open': [{'is_overnight': False, 'start': '1..."
871,44,tlDA9aMetv2D6onmJFP06w,The Raj Indian Restaurant,Restaurant,5,5.0,+442890662168,461 Lisburn Road,636.984261,"[{'open': [{'is_overnight': False, 'start': '1..."
354,18,EAzPtShIXl5vwM5bfAyEmw,Bia Rebel,Restaurant,3,5.0,+447933861720,409 Ormeau Rd,146.807286,"[{'open': [{'is_overnight': False, 'start': '1..."
917,46,0pCsvxoLIdYyYBgaxXaRDA,Gusto è,Restaurant,3,5.0,+442890333393,1 Lanyon Place,1059.657262,"[{'open': [{'is_overnight': False, 'start': '1..."
679,34,T8QDrRk0jpZHZyf4wsiH4Q,Pizza Express,Restaurant,2,5.0,+442890319000,6 St Anne's Square,1159.684098,"[{'open': [{'is_overnight': False, 'start': '1..."
145,8,edINis_jT_wEal_bYHYzHw,Little Lilly's,Restaurant,1,5.0,+442890454287,223 Castlereagh Road,103.550110,"[{'open': [{'is_overnight': False, 'start': '1..."
159,8,qrLAL4j26qfu3Hgo8A7-IA,Subway,Restaurant,1,5.0,+442890731216,436 Woodstock Rd,470.726023,"[{'open': [{'is_overnight': True, 'start': '08..."
179,9,HfS0ZFf5D07OavL9ejeG5Q,La Taqueria,Restaurant,1,5.0,+442890644477,274-276 Ormeau Road,550.944040,"[{'open': [{'is_overnight': False, 'start': '1..."
258,13,qvQ0MyItMlwjEivTHcj0GQ,Eat Street,Restaurant,1,5.0,+442890582068,231 Lisburn Rd,1139.435102,"[{'open': [{'is_overnight': False, 'start': '0..."
437,22,Ri88__kpKOZOY6nqw7sCbQ,Frankie & Benny's Boucher Place,Restaurant,1,5.0,+442890661569,1- 9 Boucher Place,829.902431,"[{'open': [{'is_overnight': False, 'start': '0..."


There are 16 5 star restaurants; and the only other way to sort them is by review count, in which case the count of ones with more than one review becomes - 

In [667]:
five_star_cats[(five_star_cats['Category']=='Restaurant') & 
               (five_star_cats['Review_Count']>1)].sort_values('Review_Count',ascending=False)

,Station_No,Business_ID,Business_Name,Category,Review_Count,Rating,Phone_No,Address,Distance,Business_Hours
243,13,lee6C8Nc-7S-fIguMu_q1w,Saphyre Restaurant,Restaurant,7,5.0,+442890688606,135 Lisburn Road,1125.207964,"[{'open': [{'is_overnight': False, 'start': '1..."
871,44,tlDA9aMetv2D6onmJFP06w,The Raj Indian Restaurant,Restaurant,5,5.0,+442890662168,461 Lisburn Road,636.984261,"[{'open': [{'is_overnight': False, 'start': '1..."
354,18,EAzPtShIXl5vwM5bfAyEmw,Bia Rebel,Restaurant,3,5.0,+447933861720,409 Ormeau Rd,146.807286,"[{'open': [{'is_overnight': False, 'start': '1..."
917,46,0pCsvxoLIdYyYBgaxXaRDA,Gusto è,Restaurant,3,5.0,+442890333393,1 Lanyon Place,1059.657262,"[{'open': [{'is_overnight': False, 'start': '1..."
679,34,T8QDrRk0jpZHZyf4wsiH4Q,Pizza Express,Restaurant,2,5.0,+442890319000,6 St Anne's Square,1159.684098,"[{'open': [{'is_overnight': False, 'start': '1..."


only 5. 